In [1]:
import numpy as np
import cv2  # OpenCVライブラリ

import matplotlib.pyplot as plt 
%matplotlib inline

import torch

import pickle

import torch.utils.data as data
from itertools import product as product

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function

In [2]:
# import dataset
from utils.dataset import VOCDataset, DatasetTransform, make_datapath_list, Anno_xml2list, od_collate_fn

In [3]:
# set your VOCdevkit path!
vocpath = "../VOCdevkit/VOC2007"
DEVKIT_PATH = "../VOCdevkit/"
SET = "test"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(vocpath)

In [4]:
len(val_img_list)

4952

In [5]:
val_anno_list[0]

'../VOCdevkit/VOC2007/Annotations/000001.xml'

In [6]:
image_index = []
for l in val_img_list:
    image_index.append(l[-10:-4])

image_index[0]

'000001'

In [7]:
class_names = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

## DatasetTransformを適応
transform = DatasetTransform(input_size, color_mean)
transform_anno = Anno_xml2list(class_names)

In [8]:
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DatasetTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(class_names))

In [9]:
val_dataloader = data.DataLoader(
    val_dataset, batch_size=1, shuffle=False, collate_fn=od_collate_fn, num_workers=8)

# set up model

In [21]:
from utils.feature_piramid_network import FPNSSD as SSD

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="inference", cfg=ssd_cfg).eval()

# SSDの学習済みの重みを設定
checkpoint = torch.load("./weights/ssd_fpn_300_200.pth")
net.load_state_dict(checkpoint["state_dict"])
#optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
print('loaded the trained weights')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using:", device)

net = net.to(device)

loaded the trained weights
using: cuda:0


In [22]:
all_imgs = []
classes = {}
bbox_threshold = 0.05

# define detections
all_boxes = [[[] for _ in range(len(val_img_list))]
               for _ in range(21)]
empty_array = np.transpose(np.array([[],[],[],[],[]]), (1,0))

In [23]:
from utils.ssd_predict_show import SSDPredictShow
ssd = SSDPredictShow(eval_categories=voc_classes, net=net, device=device)

cuda:0


# infer images

In [30]:
all_boxes = ssd.ssd_inference(val_dataloader, all_boxes, data_confidence_level=bbox_threshold)

(1, 21, 200, 5)
iter: 0
sort boxes. detection was 0.1253504753112793 and post took 0.0002505779266357422 and allboxappend took 0.007559299468994141
(1, 21, 200, 5)
iter: 1
sort boxes. detection was 0.017176151275634766 and post took 0.0003609657287597656 and allboxappend took 0.0067102909088134766
(1, 21, 200, 5)
iter: 2
sort boxes. detection was 0.05742955207824707 and post took 0.00010251998901367188 and allboxappend took 0.006819009780883789
(1, 21, 200, 5)
iter: 3
sort boxes. detection was 0.057755231857299805 and post took 8.440017700195312e-05 and allboxappend took 0.007004499435424805
(1, 21, 200, 5)
iter: 4
sort boxes. detection was 0.05150747299194336 and post took 8.988380432128906e-05 and allboxappend took 0.007034778594970703
(1, 21, 200, 5)
iter: 5
sort boxes. detection was 0.025165557861328125 and post took 7.772445678710938e-05 and allboxappend took 0.006939888000488281
(1, 21, 200, 5)
iter: 6
sort boxes. detection was 0.019189834594726562 and post took 0.000103950500488

(1, 21, 200, 5)
iter: 55
sort boxes. detection was 0.07866764068603516 and post took 0.00011444091796875 and allboxappend took 0.007371425628662109
(1, 21, 200, 5)
iter: 56
sort boxes. detection was 0.020992755889892578 and post took 8.797645568847656e-05 and allboxappend took 0.007124900817871094
(1, 21, 200, 5)
iter: 57
sort boxes. detection was 0.012172698974609375 and post took 5.7697296142578125e-05 and allboxappend took 0.006996870040893555
(1, 21, 200, 5)
iter: 58
sort boxes. detection was 0.019747257232666016 and post took 6.437301635742188e-05 and allboxappend took 0.006762504577636719
(1, 21, 200, 5)
iter: 59
sort boxes. detection was 0.1605532169342041 and post took 0.00010418891906738281 and allboxappend took 0.007166385650634766
(1, 21, 200, 5)
iter: 60
sort boxes. detection was 0.013723373413085938 and post took 6.461143493652344e-05 and allboxappend took 0.006664276123046875
(1, 21, 200, 5)
iter: 61
sort boxes. detection was 0.017413854598999023 and post took 6.818771362

(1, 21, 200, 5)
iter: 113
sort boxes. detection was 0.03223729133605957 and post took 0.00011920928955078125 and allboxappend took 0.007394313812255859
(1, 21, 200, 5)
iter: 114
sort boxes. detection was 0.03413224220275879 and post took 9.989738464355469e-05 and allboxappend took 0.007309436798095703
(1, 21, 200, 5)
iter: 115
sort boxes. detection was 0.013805627822875977 and post took 9.870529174804688e-05 and allboxappend took 0.007170200347900391
(1, 21, 200, 5)
iter: 116
sort boxes. detection was 0.01594400405883789 and post took 8.320808410644531e-05 and allboxappend took 0.0070111751556396484
(1, 21, 200, 5)
iter: 117
sort boxes. detection was 0.0373082160949707 and post took 5.984306335449219e-05 and allboxappend took 0.007106304168701172
(1, 21, 200, 5)
iter: 118
sort boxes. detection was 0.09773826599121094 and post took 0.00010395050048828125 and allboxappend took 0.007340192794799805
(1, 21, 200, 5)
iter: 119
sort boxes. detection was 0.013610601425170898 and post took 7.74

(1, 21, 200, 5)
iter: 172
sort boxes. detection was 0.024560928344726562 and post took 7.319450378417969e-05 and allboxappend took 0.0072171688079833984
(1, 21, 200, 5)
iter: 173
sort boxes. detection was 0.018352746963500977 and post took 7.05718994140625e-05 and allboxappend took 0.006894350051879883
(1, 21, 200, 5)
iter: 174
sort boxes. detection was 0.028181791305541992 and post took 0.00010585784912109375 and allboxappend took 0.007056236267089844
(1, 21, 200, 5)
iter: 175
sort boxes. detection was 0.013837814331054688 and post took 9.703636169433594e-05 and allboxappend took 0.007131338119506836
(1, 21, 200, 5)
iter: 176
sort boxes. detection was 0.011353492736816406 and post took 9.274482727050781e-05 and allboxappend took 0.00707697868347168
(1, 21, 200, 5)
iter: 177
sort boxes. detection was 0.014840126037597656 and post took 8.535385131835938e-05 and allboxappend took 0.007048606872558594
(1, 21, 200, 5)
iter: 178
sort boxes. detection was 0.016190528869628906 and post took 0

(1, 21, 200, 5)
iter: 233
sort boxes. detection was 0.014004945755004883 and post took 0.00010800361633300781 and allboxappend took 0.007180690765380859
(1, 21, 200, 5)
iter: 234
sort boxes. detection was 0.022573232650756836 and post took 6.604194641113281e-05 and allboxappend took 0.007020711898803711
(1, 21, 200, 5)
iter: 235
sort boxes. detection was 0.02373027801513672 and post took 9.584426879882812e-05 and allboxappend took 0.007189512252807617
(1, 21, 200, 5)
iter: 236
sort boxes. detection was 0.03925895690917969 and post took 0.00035881996154785156 and allboxappend took 0.006970882415771484
(1, 21, 200, 5)
iter: 237
sort boxes. detection was 0.0137939453125 and post took 9.441375732421875e-05 and allboxappend took 0.0070798397064208984
(1, 21, 200, 5)
iter: 238
sort boxes. detection was 0.01549077033996582 and post took 0.00010824203491210938 and allboxappend took 0.007086515426635742
(1, 21, 200, 5)
iter: 239
sort boxes. detection was 0.03484535217285156 and post took 9.0599

...

(1, 21, 200, 5)
iter: 4703
sort boxes. detection was 0.011392354965209961 and post took 7.891654968261719e-05 and allboxappend took 0.0070056915283203125
(1, 21, 200, 5)
iter: 4704
sort boxes. detection was 0.02572345733642578 and post took 9.083747863769531e-05 and allboxappend took 0.00709843635559082
(1, 21, 200, 5)
iter: 4705
sort boxes. detection was 0.03149533271789551 and post took 0.00010466575622558594 and allboxappend took 0.007130622863769531
(1, 21, 200, 5)
iter: 4706
sort boxes. detection was 0.09192323684692383 and post took 0.00011706352233886719 and allboxappend took 0.007155179977416992
(1, 21, 200, 5)
iter: 4707
sort boxes. detection was 0.012718677520751953 and post took 8.106231689453125e-05 and allboxappend took 0.007290840148925781
(1, 21, 200, 5)
iter: 4708
sort boxes. detection was 0.023600339889526367 and post took 9.72747802734375e-05 and allboxappend took 0.007080078125
(1, 21, 200, 5)
iter: 4709
sort boxes. detection was 0.017609119415283203 and post took 8.

(1, 21, 200, 5)
iter: 4757
sort boxes. detection was 0.03568840026855469 and post took 7.796287536621094e-05 and allboxappend took 0.007241964340209961
(1, 21, 200, 5)
iter: 4758
sort boxes. detection was 0.02246856689453125 and post took 7.796287536621094e-05 and allboxappend took 0.007090330123901367
(1, 21, 200, 5)
iter: 4759
sort boxes. detection was 0.019988298416137695 and post took 8.082389831542969e-05 and allboxappend took 0.007031679153442383
(1, 21, 200, 5)
iter: 4760
sort boxes. detection was 0.01120901107788086 and post took 6.985664367675781e-05 and allboxappend took 0.006901741027832031
(1, 21, 200, 5)
iter: 4761
sort boxes. detection was 0.017835617065429688 and post took 7.224082946777344e-05 and allboxappend took 0.006974458694458008
(1, 21, 200, 5)
iter: 4762
sort boxes. detection was 0.029422283172607422 and post took 8.368492126464844e-05 and allboxappend took 0.00711369514465332
(1, 21, 200, 5)
iter: 4763
sort boxes. detection was 0.033489227294921875 and post too

(1, 21, 200, 5)
iter: 4818
sort boxes. detection was 0.04578685760498047 and post took 0.00010895729064941406 and allboxappend took 0.007268667221069336
(1, 21, 200, 5)
iter: 4819
sort boxes. detection was 0.044643402099609375 and post took 0.00010251998901367188 and allboxappend took 0.007185459136962891
(1, 21, 200, 5)
iter: 4820
sort boxes. detection was 0.02072453498840332 and post took 8.96453857421875e-05 and allboxappend took 0.007015228271484375
(1, 21, 200, 5)
iter: 4821
sort boxes. detection was 0.019532203674316406 and post took 0.00010275840759277344 and allboxappend took 0.007060527801513672
(1, 21, 200, 5)
iter: 4822
sort boxes. detection was 0.012188196182250977 and post took 9.441375732421875e-05 and allboxappend took 0.007100582122802734
(1, 21, 200, 5)
iter: 4823
sort boxes. detection was 0.07512068748474121 and post took 8.749961853027344e-05 and allboxappend took 0.0072247982025146484
(1, 21, 200, 5)
iter: 4824
sort boxes. detection was 0.04918408393859863 and post 

(1, 21, 200, 5)
iter: 4874
sort boxes. detection was 0.023891448974609375 and post took 0.0003533363342285156 and allboxappend took 0.006842613220214844
(1, 21, 200, 5)
iter: 4875
sort boxes. detection was 0.06141066551208496 and post took 0.00010132789611816406 and allboxappend took 0.00719761848449707
(1, 21, 200, 5)
iter: 4876
sort boxes. detection was 0.015406370162963867 and post took 7.724761962890625e-05 and allboxappend took 0.007031917572021484
(1, 21, 200, 5)
iter: 4877
sort boxes. detection was 0.03992128372192383 and post took 0.00011777877807617188 and allboxappend took 0.007225513458251953
(1, 21, 200, 5)
iter: 4878
sort boxes. detection was 0.016826868057250977 and post took 0.00010418891906738281 and allboxappend took 0.0071523189544677734
(1, 21, 200, 5)
iter: 4879
sort boxes. detection was 0.09170389175415039 and post took 0.0001087188720703125 and allboxappend took 0.007334709167480469
(1, 21, 200, 5)
iter: 4880
sort boxes. detection was 0.01142120361328125 and post 

(1, 21, 200, 5)
iter: 4932
sort boxes. detection was 0.020871400833129883 and post took 7.581710815429688e-05 and allboxappend took 0.007157325744628906
(1, 21, 200, 5)
iter: 4933
sort boxes. detection was 0.02824258804321289 and post took 0.00010800361633300781 and allboxappend took 0.007189750671386719
(1, 21, 200, 5)
iter: 4934
sort boxes. detection was 0.01798558235168457 and post took 0.00010228157043457031 and allboxappend took 0.007103443145751953
(1, 21, 200, 5)
iter: 4935
sort boxes. detection was 0.01566457748413086 and post took 9.036064147949219e-05 and allboxappend took 0.007033109664916992
(1, 21, 200, 5)
iter: 4936
sort boxes. detection was 0.04848980903625488 and post took 0.00010514259338378906 and allboxappend took 0.007066011428833008
(1, 21, 200, 5)
iter: 4937
sort boxes. detection was 0.012304544448852539 and post took 6.628036499023438e-05 and allboxappend took 0.006906032562255859
(1, 21, 200, 5)
iter: 4938
sort boxes. detection was 0.016267061233520508 and post 

In [31]:
all_boxes[7][0:10]

[array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 [[0.9998599, 205.93579, 250.33597, 290.82626, 292.5944],
  [0.99978787, 7.193805, 218.56204, 49.24107, 256.79718],
  [0.99959356, 240.67632, 252.62904, 325.7241, 295.59174],
  [0.99908185, 7.073488, 260.81418, 50.168163, 300.9781],
  [0.98482805, 140.2719, 236.32469, 202.81073, 269.7898],
  [0.9539366, 114.904594, 273.1905, 155.43486, 299.82895],
  [0.94417757, 113.77047, 260.81055, 158.02158, 287.9876],
  [0.9120234, 94.65097, 268.7511, 136.80014, 295.24646],
  [0.8857979, 109.490654, 212.28517, 152.65985, 242.1866],
  [0.8853462, 145.45047, 216.76001, 209.58931, 253.99007],
  [0.86547565, 71.03111, 234.23355, 101.50582, 262.40997],
  [0.8566358, 88.26479, 255.19589, 133.09476, 283.46396],
  [0.8494147, 179.14442, 252.40277, 257.79276, 295.1969],
  [0.8175253, 89.84482, 231.40204, 119.1359, 258.9404],
  [0.80859596, 7.125438, 172.5301, 48.800694, 211.13152],
  [0

# eval accuracy

In [24]:
# eval function
def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             cachedir,
             ovthresh=0.5,
             use_07_metric=False):
  """
  rec, prec, ap = voc_eval(detpath,
                              annopath,
                              imagesetfile,
                              classname,
                              [ovthresh],
                              [use_07_metric])
  Top level function that does the PASCAL VOC evaluation.
  detpath: Path to detections
      detpath.format(classname) should produce the detection results file.
  annopath: Path to annotations
      annopath.format(imagename) should be the xml annotations file.
  imagesetfile: Text file containing the list of images, one image per line.
  classname: Category name (duh)
  cachedir: Directory for caching the annotations
  [ovthresh]: Overlap threshold (default = 0.5)
  [use_07_metric]: Whether to use VOC07's 11 point AP computation
      (default False)
  """
  # assumes detections are in detpath.format(classname)
  # assumes annotations are in annopath.format(imagename)
  # assumes imagesetfile is a text file with each line an image name
  # cachedir caches the annotations in a pickle file

  # first load gt
  if not os.path.isdir(cachedir):
    os.mkdir(cachedir)
  cachefile = os.path.join(cachedir, '%s_annots.pkl' % imagesetfile)
  # read list of images
  with open(imagesetfile, 'r') as f:
    lines = f.readlines()
  imagenames = [x.strip() for x in lines]

  if not os.path.isfile(cachefile):
    # load annotations
    recs = {}
    for i, imagename in enumerate(imagenames):
      recs[imagename] = parse_rec(annopath.format(imagename))
      if i % 100 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
    # save
    #print('Saving cached annotations to {:s}'.format(cachefile))
    #with open(cachefile, 'wb') as f:
    #  pickle.dump(recs, f)
  else:
    # load
    with open(cachefile, 'rb') as f:
      try:
        recs = pickle.load(f)
      except:
        recs = pickle.load(f, encoding='bytes')

  # extract gt objects for this class
  class_recs = {}
  npos = 0
  for imagename in imagenames:
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    npos = npos + sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
                             'difficult': difficult,
                             'det': det}

  # read dets
  detfile = detpath.format(classname)
  with open(detfile, 'r') as f:
    lines = f.readlines()

  splitlines = [x.strip().split(' ') for x in lines]
  image_ids = [x[0] for x in splitlines]
  confidence = np.array([float(x[1]) for x in splitlines])
  BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

  nd = len(image_ids)
  tp = np.zeros(nd)
  fp = np.zeros(nd)

  if BB.shape[0] > 0:
    # sort by confidence
    sorted_ind = np.argsort(-confidence)
#    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    for d in range(nd):
      id = image_ids[d][-10:-4]
      #print(id)
      # catch bad detections
      try:
          R = class_recs[id]
      except:
        print("det not found")
        continue
        
      bb = BB[d, :].astype(float)
      ovmax = -np.inf
      BBGT = R['bbox'].astype(float)

      if BBGT.size > 0:
        # compute overlaps
        # intersection
        ixmin = np.maximum(BBGT[:, 0], bb[0])
        iymin = np.maximum(BBGT[:, 1], bb[1])
        ixmax = np.minimum(BBGT[:, 2], bb[2])
        iymax = np.minimum(BBGT[:, 3], bb[3])
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)
        inters = iw * ih

        # union
        uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
               (BBGT[:, 2] - BBGT[:, 0] + 1.) *
               (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

        overlaps = inters / uni
        ovmax = np.max(overlaps)
        jmax = np.argmax(overlaps)

      if ovmax > ovthresh:
        if not R['difficult'][jmax]:
          if not R['det'][jmax]:
            tp[d] = 1.
            R['det'][jmax] = 1
          else:
            fp[d] = 1.
      else:
        fp[d] = 1.

  # compute precision recall
  fp = np.cumsum(fp)
  tp = np.cumsum(tp)
  rec = tp / float(npos)
  # avoid divide by zero in case the first detection matches a difficult
  # ground truth
  prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
  ap = voc_ap(rec, prec, use_07_metric)

  return rec, prec, ap

In [25]:
pascal_classes = np.asarray(["__background__", 'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor'])

PASCAL_CLASSES = pascal_classes

# write out detections for evaluation

In [26]:
import os 
def get_voc_results_file_template(cls):
        # VOCdevkit/results/VOC2007/Main/<comp_id>_det_test_aeroplane.txt
        filename = 'det_' + "val" + '_'+cls+'.txt'
        filedir = os.path.join(DEVKIT_PATH, 'results', 'VOC2007', 'Main')
        if not os.path.exists(filedir):
            os.makedirs(filedir)
        path = os.path.join(filedir, filename)
        return path


def write_voc_results_file(pascal_classes, all_boxes, image_index):
        for cls_ind, cls in enumerate(pascal_classes):
            if cls == '__background__':
                continue
            print('Writing {} VOC results file'.format(cls))
            filename = get_voc_results_file_template(cls)
            with open(filename, 'wt') as f:
                for im_ind, index in enumerate(image_index):
                    dets = np.asarray(all_boxes[cls_ind][im_ind])
                    if dets == []:
                        continue
                    # the VOCdevkit expects 1-based indices
                    for k in range(dets.shape[0]):
                        #print(dets[k, 0])
                        f.write('{:s} {:.3f} {:.1f} {:.1f} {:.1f} {:.1f}\n'.
                                format(index, dets[k, 0],
                                       dets[k, 1] + 1, dets[k, 2] + 1,
                                       dets[k, 3] + 1, dets[k, 4] + 1))
import xml.etree.ElementTree as ET
def parse_rec(filename):
  """ Parse a PASCAL VOC xml file """
  tree = ET.parse(filename)
  objects = []
  for obj in tree.findall('object'):
    obj_struct = {}
    obj_struct['name'] = obj.find('name').text
    obj_struct['pose'] = obj.find('pose').text
    obj_struct['truncated'] = int(obj.find('truncated').text)
    obj_struct['difficult'] = int(obj.find('difficult').text)
    bbox = obj.find('bndbox')
    obj_struct['bbox'] = [int(bbox.find('xmin').text),
                          int(bbox.find('ymin').text),
                          int(bbox.find('xmax').text),
                          int(bbox.find('ymax').text)]
    objects.append(obj_struct)

  return objects
def voc_ap(rec, prec, use_07_metric=False):
  """ ap = voc_ap(rec, prec, [use_07_metric])
  Compute VOC AP given precision and recall.
  If use_07_metric is true, uses the
  VOC 07 11 point method (default:False).
  """
  if use_07_metric:
    # 11 point metric
    ap = 0.
    for t in np.arange(0., 1.1, 0.1):
      if np.sum(rec >= t) == 0:
        p = 0
      else:
        p = np.max(prec[rec >= t])
      ap = ap + p / 11.
  else:
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
      mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
  return ap

In [27]:
write_voc_results_file(pascal_classes, all_boxes, val_img_list)

Writing aeroplane VOC results file
Writing bicycle VOC results file
Writing bird VOC results file
Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC results file
Writing horse VOC results file
Writing motorbike VOC results file
Writing person VOC results file
Writing pottedplant VOC results file
Writing sheep VOC results file
Writing sofa VOC results file
Writing train VOC results file
Writing tvmonitor VOC results file


# evaluation

In [28]:
def python_eval(output_dir='output'):
        annopath = os.path.join(
            DEVKIT_PATH,
            'VOC2007',
            'Annotations',
            '{:s}.xml')
        imagesetfile = os.path.join(
            DEVKIT_PATH,
            'VOC2007',
            'ImageSets',
            'Main',
            SET + '.txt')
        cachedir = os.path.join(DEVKIT_PATH, 'annotations_cache')
        aps = []
        # The PASCAL VOC metric changed in 2010.
        # VOC07 metric is quite old so don't use.
        use_07_metric = False
        print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
        for i, cls in enumerate(PASCAL_CLASSES):
            if cls == '__background__':
                continue
            filename = get_voc_results_file_template(cls)
            rec, prec, ap = voc_eval(
                filename, annopath, imagesetfile, cls, cachedir, ovthresh=0.5,
                use_07_metric=use_07_metric)
            aps += [ap]
            print('AP for {} = {:.4f}'.format(cls, ap))
            with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
                pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
        print('Mean AP = {:.4f}'.format(np.mean(aps)))
        print('~~~~~~~~')
        print('Results:')
        for ap in aps:
            print('{:.3f}'.format(ap))
        print('{:.3f}'.format(np.mean(aps)))
        print('~~~~~~~~')
        print('')
        print('--------------------------------------------------------------')
        print('Results computed with the **unofficial** Python eval code.')
        print('Results should be very close to the official MATLAB eval code.')
        print('Recompute with `./tools/reval.py --matlab ...` for your paper.')
        print('-- Thanks, The Management')
        print('--------------------------------------------------------------')

In [29]:
# evaluate detections
python_eval()

VOC07 metric? No
Reading annotation for 1/4952
Reading annotation for 101/4952
Reading annotation for 201/4952
Reading annotation for 301/4952
Reading annotation for 401/4952
Reading annotation for 501/4952
Reading annotation for 601/4952
Reading annotation for 701/4952
Reading annotation for 801/4952
Reading annotation for 901/4952
Reading annotation for 1001/4952
Reading annotation for 1101/4952
Reading annotation for 1201/4952
Reading annotation for 1301/4952
Reading annotation for 1401/4952
Reading annotation for 1501/4952
Reading annotation for 1601/4952
Reading annotation for 1701/4952
Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Readi

Reading annotation for 1301/4952
Reading annotation for 1401/4952
Reading annotation for 1501/4952
Reading annotation for 1601/4952
Reading annotation for 1701/4952
Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Reading annotation for 3001/4952
Reading annotation for 3101/4952
Reading annotation for 3201/4952
Reading annotation for 3301/4952
Reading annotation for 3401/4952
Reading annotation for 3501/4952
Reading annotation for 3601/4952
Reading annotation for 3701/4952
Reading annotation for 3801/4952
Reading annotation for 3901/4952
Reading annotation for 4001/4952
Reading annotation for 4101/4952
Reading annotation for 4201/4952
Reading an

Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Reading annotation for 3001/4952
Reading annotation for 3101/4952
Reading annotation for 3201/4952
Reading annotation for 3301/4952
Reading annotation for 3401/4952
Reading annotation for 3501/4952
Reading annotation for 3601/4952
Reading annotation for 3701/4952
Reading annotation for 3801/4952
Reading annotation for 3901/4952
Reading annotation for 4001/4952
Reading annotation for 4101/4952
Reading annotation for 4201/4952
Reading annotation for 4301/4952
Reading annotation for 4401/4952
Reading annotation for 4501/4952
Reading annotation for 4601/4952
Reading annotation for 4701/4952
Reading an

Reading annotation for 3501/4952
Reading annotation for 3601/4952
Reading annotation for 3701/4952
Reading annotation for 3801/4952
Reading annotation for 3901/4952
Reading annotation for 4001/4952
Reading annotation for 4101/4952
Reading annotation for 4201/4952
Reading annotation for 4301/4952
Reading annotation for 4401/4952
Reading annotation for 4501/4952
Reading annotation for 4601/4952
Reading annotation for 4701/4952
Reading annotation for 4801/4952
Reading annotation for 4901/4952
AP for pottedplant = 0.0000
Reading annotation for 1/4952
Reading annotation for 101/4952
Reading annotation for 201/4952
Reading annotation for 301/4952
Reading annotation for 401/4952
Reading annotation for 501/4952
Reading annotation for 601/4952
Reading annotation for 701/4952
Reading annotation for 801/4952
Reading annotation for 901/4952
Reading annotation for 1001/4952
Reading annotation for 1101/4952
Reading annotation for 1201/4952
Reading annotation for 1301/4952
Reading annotation for 1401